In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

### Preparación de datos

In [4]:
df_esios = pd.read_csv('data_training/esios_previsiones_d+1.csv')
df_esios['Datetime'] = pd.to_datetime(df_esios['Date']) + pd.to_timedelta(df_esios['Hour'] - 1, unit='h')
df_esios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65592 entries, 0 to 65591
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   indicator_id  65592 non-null  int64         
 1   Date          65592 non-null  object        
 2   Hour          65592 non-null  int64         
 3   geo_id        65592 non-null  int64         
 4   value         65592 non-null  float64       
 5   Datetime      65592 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(3), object(1)
memory usage: 3.0+ MB


In [6]:
df_esios_clean = df_esios[['Datetime', 'indicator_id', 'value']].copy()
df_esios_clean.drop_duplicates(subset=['Datetime', 'indicator_id'], keep='first', inplace=True)

df_esios_pivot = df_esios_clean.pivot(
    index='Datetime',
    columns='indicator_id',
    values='value'
).reset_index()

df_esios_pivot

indicator_id,Datetime,1775,1777,1779
0,2023-01-01 00:00:00,20059.8,10266.8,0.0
1,2023-01-01 01:00:00,19380.3,10116.5,0.0
2,2023-01-01 02:00:00,18523.8,9954.0,0.0
3,2023-01-01 03:00:00,17588.3,9816.0,0.0
4,2023-01-01 04:00:00,16834.5,9625.5,0.0
...,...,...,...,...
21857,2025-06-29 20:00:00,31241.5,4842.3,4534.8
21858,2025-06-29 21:00:00,32164.5,4481.8,632.8
21859,2025-06-29 22:00:00,32269.3,4226.0,0.0
21860,2025-06-29 23:00:00,30043.3,3804.3,0.0


In [13]:
df_input = df_esios_pivot.copy()

df_input['Year'] = df_input['Datetime'].dt.year
df_input['Month'] = df_input['Datetime'].dt.month
df_input['Day_of_Week'] = df_input['Datetime'].dt.dayofweek
df_input['Hour'] = df_input['Datetime'].dt.hour

dicc_indicators = {1775: 'demanda', 1777: 'gen_eolica', 1779: 'gen_fotovoltaica'}

df_input.rename(columns=dicc_indicators, inplace=True)
df_input

indicator_id,Datetime,demanda,gen_eolica,gen_fotovoltaica,Year,Month,Day_of_Week,Hour
0,2023-01-01 00:00:00,20059.8,10266.8,0.0,2023,1,6,0
1,2023-01-01 01:00:00,19380.3,10116.5,0.0,2023,1,6,1
2,2023-01-01 02:00:00,18523.8,9954.0,0.0,2023,1,6,2
3,2023-01-01 03:00:00,17588.3,9816.0,0.0,2023,1,6,3
4,2023-01-01 04:00:00,16834.5,9625.5,0.0,2023,1,6,4
...,...,...,...,...,...,...,...,...
21857,2025-06-29 20:00:00,31241.5,4842.3,4534.8,2025,6,6,20
21858,2025-06-29 21:00:00,32164.5,4481.8,632.8,2025,6,6,21
21859,2025-06-29 22:00:00,32269.3,4226.0,0.0,2025,6,6,22
21860,2025-06-29 23:00:00,30043.3,3804.3,0.0,2025,6,6,23
